In [1]:
import os
PATH = os.path.realpath(os.path.dirname(__file__))

# config constant
CONFIG_FILE_PATH_PREFIX = PATH + "/../cfg/"
CONFIG_TMP_FILE_PATH_PREFIX = PATH + "/../cfg/tmp/"
CONFIG_BACKUP_FILE_PATH_PREFIX = PATH + "/../cfg/backup/"

OUTPUT_FILE_PATH_PREFIX = PATH + "/../output/"
OUTPUT_TMP_FILE_PATH_PREFIX = PATH + "/../output/tmp/"

CONFIG_NUMBER = 1000

In [2]:
#!pip install Faker

In [3]:
import sys
import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType,StructField, StringType
spark = SparkSession \
    .builder \
    .appName("Python Spark Customer Fact Generator") \
    .getOrCreate()

23/01/05 11:12:49 WARN Utils: Your hostname, htknphuong-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.142.129 instead (on interface ens33)
23/01/05 11:12:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/05 11:12:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
##Reading the config from file for max id
import os.path
idConfig = {
    "accountrecordid": 10000000,
    "defaultaccountrecordid": 10000000,
    "accountid": 10000000,
    "defaultaccountid": 10000000,
    "defaultcustomerid": 10000000,
    "customerid": 10000000,
}
print('-----------------')
for file in idConfig.keys():
    file_exist = os.path.exists(CONFIG_FILE_PATH_PREFIX+file)
    if (file_exist):
        print(file + ' is exist')
        print('begin loading' + file)
        realFile = open(CONFIG_FILE_PATH_PREFIX+file,"r")
        idConfig[file] = int(realFile.readlines()[0])
        realFile.close()
    else:
        if (file not in ["accountid", "defaultaccountid", "accountrecordid", "defaultaccountrecordid"]):
            print ('Please run CustomerFactGenerator first')
            break
        print(file + ' is not exist')
        print('begin to creating ' + file)
        realFile = open(CONFIG_FILE_PATH_PREFIX+file,"w+")
        realFile.seek(0)
        realFile.write(str(idConfig[file]))
        realFile.close()

-----------------
accountrecordid is exist
begin loadingaccountrecordid
defaultaccountrecordid is exist
begin loadingdefaultaccountrecordid
accountid is exist
begin loadingaccountid
defaultaccountid is exist
begin loadingdefaultaccountid
defaultcustomerid is exist
begin loadingdefaultcustomerid
customerid is exist
begin loadingcustomerid


In [5]:
print(idConfig)

{'accountrecordid': 10001004, 'defaultaccountrecordid': 10000000, 'accountid': 10013004, 'defaultaccountid': 10000000, 'defaultcustomerid': 10002000, 'customerid': 10003000}


In [13]:
import random
import faker
from datetime import datetime
import re
class AccountObject:
    AcountTypeDim = {
        1: "credit",
        2: "debit",
        3: "deposit",
        4: "saving"
    }
    StatusDim = {
        1: "Active",
        2: "Inactive",
        3: "Deleted",
        4: "OnewayInactive"
    }
    customerType = {
        1: "Company",
        2: "Invidual",
        3: "LinkedCompanyUser"
    }
    fake = faker.Faker()

    def __init__(self):
        self.accountrecordid = idConfig["accountrecordid"]
        idConfig["accountrecordid"] += 1
        self.accountid = idConfig["accountid"]
        idConfig["accountid"] += 1
        __accounttype = random.choice(list(AccountObject.AcountTypeDim.keys()))
        self.accounttype = AccountObject.AcountTypeDim[__accounttype]
        self.customerType = AccountObject.customerType[2]
        self.customerid = AccountObject.fake.random_int(min=idConfig['defaultcustomerid'], max=idConfig['customerid'])
        self.balance = random.randint(1, 10000)
        self.opendate = AccountObject.fake.date()
        self.closedate = ""
        self.duration = ""
        self.initialamount = ""
        self.remainamount = ""
        self.totalamount = ""
        self.status = AccountObject.StatusDim[1]
        self.loanDate = ""
        self.loanTypeID = ""
        self.referenceinterestrate = ""
        self.interestID = ""
        self.cardinfo = AccountObject.fake.credit_card_full()

    def toList(self):
        return [Row(
            self.accountrecordid,
            self.accountid,
            self.accounttype,
            self.customerType,
            self.customerid,
            self.balance,
            self.opendate,
            self.closedate,
            self.duration,
            self.initialamount,
            self.remainamount,
            self.totalamount,
            self.status,
            self.loanDate,
            self.loanTypeID,
            self.referenceinterestrate,
            self.interestID,
            self.cardinfo
            )]


In [14]:
FactAccount = []
for x in range(CONFIG_NUMBER):
    Account = AccountObject().toList()
    FactAccount = FactAccount + Account

In [18]:
FactAccountColumns = StructType([
    StructField("accountrecordid", StringType(), True),
    StructField("accountid", StringType(), True),
    StructField("accounttype", StringType(), True),
    StructField("customerType", StringType(), True),
    StructField("customerid", StringType(), True),
    StructField("balance", StringType(), True),
    StructField("opendate", StringType(), True),
    StructField("closedate", StringType(), True),
    StructField("duration", StringType(), True),
    StructField("initialamount", StringType(), True),
    StructField("remainamount", StringType(), True),
    StructField("totalamount", StringType(), True),
    StructField("status", StringType(), True),
    StructField("loanDate", StringType(), True),
    StructField("loanTypeID", StringType(), True),
    StructField("referenceinterestrate", StringType(), True),
    StructField("interestID", StringType(), True),
    StructField("cardinfo", StringType(), True),
    ])

In [19]:
MainDf = spark.createDataFrame(data=FactAccount, schema = FactAccountColumns)

In [20]:
from datetime import datetime
now = datetime.now()
fileName = 'FactAccount ' + now.strftime("%Y-%m-%d %H-%M-%S-%f") 

In [21]:
print(fileName)

FactAccount 2023-01-05 11-16-48-188542


In [22]:
MainDf.write.mode('overwrite').option("header","true").csv(OUTPUT_TMP_FILE_PATH_PREFIX + fileName)

In [23]:
for file in idConfig.keys():
    print('begin to rewrite ' + file)
    realFile = open(CONFIG_TMP_FILE_PATH_PREFIX+file,"w+")
    realFile.seek(0)
    realFile.write(str(idConfig[file]))
    realFile.close()
    print('-----------------')

begin to rewrite accountrecordid
-----------------
begin to rewrite defaultaccountrecordid
-----------------
begin to rewrite accountid
-----------------
begin to rewrite defaultaccountid
-----------------
begin to rewrite defaultcustomerid
-----------------
begin to rewrite customerid
-----------------


In [24]:
import shutil
for file in idConfig.keys():
    shutil.move(CONFIG_FILE_PATH_PREFIX+file, CONFIG_BACKUP_FILE_PATH_PREFIX+file)
    shutil.move(CONFIG_TMP_FILE_PATH_PREFIX+file, CONFIG_FILE_PATH_PREFIX+file)
shutil.move(OUTPUT_TMP_FILE_PATH_PREFIX + fileName, OUTPUT_FILE_PATH_PREFIX+fileName)

'output/FactAccount 2023-01-05 11-16-48-188542'